In [ ]:
# Įdiekite reikiamas bibliotekas
!pip install gTTS kornia
!apt update
!apt install ffmpeg

# Clone SadTalker repository
!git clone https://github.com/cedro3/SadTalker.git
%cd SadTalker

!pip install -r requirements.txt

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from gtts import gTTS
import IPython.display as ipd
import torch
import os
import glob
from base64 import b64encode
from IPython.display import display, HTML

# T5-Small model
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

# Text to summarize
text = "summarize: " + """
"Politics is a crucial aspect of society that influences our lives, rights, and responsibilities. It encompasses decisions and actions aimed at governing a country or its parts. Politics affects various areas, from education and healthcare to the economy and the environment.

In democratic countries, politics is often based on the opinions of the people.
"""

# Encode the text
inputs = tokenizer.encode(text, return_tensors="pt")

# Generate summary with the model
outputs = model.generate(inputs, max_length=2, num_beams=5, length_penalty=1.0)

# Decode the output
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Check if the file exists
if os.path.exists('/content/summary.mp3'):
    # If so, remove it
    os.remove('/content/summary.mp3')

# Convert the summary to speech
tts = gTTS(summary, lang='en')
tts.save('/content/summary.mp3')

# Play the audio file
ipd.Audio('/content/summary.mp3')

# Download pre-trained models
!bash scripts/download_models.sh

# Inference for face
image ='full3.png' # replace with your image file
audio = '/content/summary.mp3' # use the generated audio file

source_image = 'examples/source_image/' + image
driven_audio = audio # use the generated audio file directly

# Similarly for the video file
files = glob.glob('./results/*.mp4')
for f in files:
    os.remove(f)

# Run the inference
!python3 inference.py --driven_audio $driven_audio \
                      --source_image $source_image \
                      --result_dir ./results --enhancer gfpgan \
                      --cpu

# Get the list of .mp4 files in the ./results/ directory
mp4_files = [file for file in os.listdir('./results/') if file.endswith('.mp4')]

# Check if the list is not empty
if mp4_files:
    # If so, get the path of the first video
    video_path = './results/' + mp4_files[0]

    # Convert the video to HTML format
    if os.path.exists(video_path):
        mp4 = open(video_path,'rb').read()
        data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
        display(HTML("""
        <video width=400 controls>
              <source src="%s" type="video/mp4">
        </video>
        """ % data_url))
    else:
        print("The video file does not exist")
else:
    print("No .mp4 files found in ./results/ directory")


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
28 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
f